# Data Science in de Cloud: De "Azure ML SDK" aanpak

## Introductie

In dit notebook leren we hoe we de Azure ML SDK kunnen gebruiken om een model te trainen, implementeren en gebruiken via Azure ML.

Voorwaarden:
1. Je hebt een Azure ML-werkruimte aangemaakt.
2. Je hebt de [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) geladen in Azure ML.
3. Je hebt dit notebook geüpload naar Azure ML Studio.

De volgende stappen zijn:

1. Maak een Experiment in een bestaande Werkruimte.
2. Maak een Compute-cluster.
3. Laad de dataset.
4. Configureer AutoML met behulp van AutoMLConfig.
5. Voer het AutoML-experiment uit.
6. Verken de resultaten en selecteer het beste model.
7. Registreer het beste model.
8. Implementeer het beste model.
9. Gebruik de endpoint.

## Azure Machine Learning SDK-specifieke imports


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Werkruimte Initialiseren
Initialiseer een werkruimte-object vanuit opgeslagen configuratie. Zorg ervoor dat het configuratiebestand aanwezig is op .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Een Azure ML-experiment maken

Laten we een experiment genaamd 'aml-experiment' maken in de werkruimte die we zojuist hebben geïnitialiseerd.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Een Compute Cluster maken
Je moet een [compute-doel](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) aanmaken voor je AutoML-run.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Gegevens
Zorg ervoor dat je de dataset hebt geüpload naar Azure ML en dat de sleutel dezelfde naam heeft als de dataset.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML-configuratie


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## AutoML Run


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Beste model implementeren

Voer de volgende code uit om het beste model te implementeren. Je kunt de status van de implementatie bekijken in het Azure ML-portaal. Deze stap kan enkele minuten duren.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Gebruik de Eindpunt
Je kunt invoer toevoegen aan het volgende invoervoorbeeld.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, willen we u erop wijzen dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
